# Get Data

In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import numpy as np 

key_path = f"../../secrets/google_creds.json"
credentials = service_account.Credentials.from_service_account_file(key_path,
                                                                    scopes=[
                                                                        "https://www.googleapis.com/auth/cloud-platform"], )
client = bigquery.Client(credentials=credentials, project=credentials.project_id)
query = """SELECT * FROM
`net_expr.trials` AS t
INNER JOIN `net_expr.person` as p
ON t.participant_id = p.participant_id
WHERE 
p.is_test is FALSE AND p.participant_id != 'seed' 
"""
query_job = client.query(query)
df = query_job.to_dataframe()

# Two cases where a test response (response_text == "Test") was not flagged --> drop, note in paper
er_ids = ['dbe96241-4d64-4ae6-bb6d-15e5e45b0760', 'e6fdd18f-de53-4159-965a-656aa75ba88d']
df = df.query("response_id not in @er_ids")

FileNotFoundError: [Errno 2] No such file or directory: '../../secrets/google_creds.json'

## Data Checks

In [ ]:
def verify_rid_unique():
    print("TEST: Does every response id occur only once?")
    unique_rds = len(df['response_id'].unique())
    all_rids = len(df['response_id'].tolist())
    if all_rids == unique_rds:
        print("YES: Every response id only occurs once")
    else:
        print("NO: Some response ids occur twice")
    
    
def verify_no_response_blank():
    print("TEST: Is it the case that all response texts have some words")
    blanks = len([x for x in df['response_id'].tolist() if x is None])
    if blanks == 0:
        print("YES: All responses have some words")
    else:
        print("NO: Some responses don't have words")
    
        
verify_rid_unique()
print()
verify_no_response_blank()

# Clean Data

## Clean source column

We tracked where traffic came from by appending different things to the request arguments for the experiment site. E.g: If posting the experiment on Facebook we would make the URL `experiment.com?from=facebook` so then `request_args` would be equal to `from=facebook`. But in some cases, no `request args` are available. This happens if somehow the request_args are stripped. There should be very few cases of this. 

In [ ]:
def verify_other_source():
    print("TEST: Does every source marked as OTHER have no request args?")
    other_request_args = df.query("source=='other'")['request_args'].tolist()
    len_other = len(other_request_args)
    len_other_no_args = len([x for x in other_request_args if x == 'None'])
    if len_other == len_other_no_args:
        print("YES: For every source marked as `other`, we marked it as `other` because it does not have request args")
    else:
        print("NO: We missed categorizing some sources")
    
    

def categorize_request_args(request_args):
    request_args = request_args.lower()
    if 'facebook' in request_args:
        return 'facebook'
    elif 'chatgpt' in request_args:
        return 'r/chatgpt'
    elif 'sample' in request_args:
        return 'r/samplesize'
    elif 'writing' in request_args:
        return 'r/writing'
    elif 'internet' in request_args:
        return 'r/InternetIsBeautiful'
    elif 'creative' in request_args:
        return 'Creative Mornings newsletter'
    elif 'poetry' in request_args:
        return 'r/poetry'
    elif 'results' in request_args or 'share' in request_args:
        return 'share'
    else:
        return 'other'

df['source'] = df['request_args'].apply(categorize_request_args)
verify_other_source()

df['source'].value_counts()

## Clean Age Columns
Participants entered their age, which was an int that had to be over 18. But let's make sure nobody entered anything weird -- and by weird we will look at responses over 70 years old. From inspecting the data, it appears that responses above 74 are troll responses -- relabel as NA and report in paper. 

In [ ]:
dfp = df.drop_duplicates(subset=['participant_id'])
dfp['age'].describe()

In [ ]:
over_seven = dfp[dfp['age']>=70]
over_seven

In [ ]:
print(dfp[dfp['age']>74][['participant_id', 'age']].to_latex(caption="Users whose age we replaced with missing"))

In [ ]:
def fix_age(x):
    if pd.isnull(x):  # Check for NaN values
        return x
    elif x <= 74:
        return int(x)
    else:
        return np.NaN

df['age'] = df['age'].apply(lambda x: fix_age(x))
df['age'] = df['age'].astype('Int64')

# Clean columns

## Drop unused columns

Note `is_troll` was not used; the relevant column is `is_profane`. 

In [ ]:
df.columns

In [ ]:
to_drop = ["is_prolific", "is_prolific_1", "prolific_id", "is_test_1", "is_troll", "dt", 'participant_id_1']
for x in to_drop:
    try:
        df = df.drop(columns = [x], axis=0)
    except KeyError:
        pass

## Fix column types and names

In [ ]:
df['creativity_human'] = df['creativity_human'].astype('Int64')
df['creativity_ai'] = pd.to_numeric(df['creativity_ai'], errors='coerce').astype('Int64')
df = df.rename(columns={'world': 'response_chain'})

# Add trial_no

In [ ]:
df['response_date'] = pd.to_datetime(df['response_date'])  # ensure response_date is datetime
df['trial_no'] = df.groupby(['response_chain', 'condition', 'item'])['response_date'].rank(method='first').astype(int)

## Save data

In [ ]:
df.to_csv("../../data/experiment_data/data_clean.csv")

# Descriptive Stats

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
def make_aesthetic(hex_color_list=None):
    """Make Seaborn look clean"""
    sns.set(style='white', context='poster', font_scale=0.8)
    if not hex_color_list:
        hex_color_list = [
        "#826AED", # Medium slate blue
        "#00A896", # Persian green
        "#D41876", # Telemagenta
        "#89DAFF", # Pale azure
        "#F7B2AD", # Melon
        "#342E37", # Dark grayish-purple
        "#7DCD85", # Emerald
        "#E87461", # Medium-bright orange
        "#E3B505", # Saffron
        "#2C3531", # Dark charcoal gray with a green undertone
        "#D4B2D8", # Pink lavender
        "#7E6551", # Coyote
        "#F45B69", # Vibrant pinkish-red
        "#020887", # Phthalo Blue
        "#F18805"  # Tangerine
        ]
    sns.set_palette(sns.color_palette(hex_color_list))
    plt.rcParams['font.family'] = 'Arial'
    plt.rcParams['axes.spines.right'] = False
    plt.rcParams['axes.spines.top'] = False
    plt.rcParams['axes.titlelocation'] = 'left'
    plt.rcParams['axes.titleweight'] = 'bold'
    plt.rcParams['axes.grid'] = True
    plt.rcParams['grid.linestyle'] = '--'
    plt.rcParams['grid.alpha'] = 0.5
    plt.rcParams['axes.facecolor'] = 'white'
    plt.rcParams['legend.frameon'] = True
    plt.rcParams['legend.framealpha'] = 0.8
    plt.rcParams['legend.facecolor'] = 'white'
    plt.rcParams['savefig.transparent'] = True
    plt.rcParams['savefig.bbox'] = 'tight'
    plt.rcParams['savefig.pad_inches'] = 0.1
    plt.rcParams['figure.autolayout'] = True
make_aesthetic()

### Meta Info

In [ ]:
df_new = pd.DataFrame(index=["Value"])
df_new['Unique Countries'] = [int(len(df['country'].unique()))]
df_new['Total Responses'] = [int(len(df))]
df_new['Unique Participants'] = [df['participant_id'].nunique()]
df_new['Avg Responses/Participant'] = df_new['Total Responses'] / df_new['Unique Participants']
df_new['Avg Duration/Response'] = [df['duration'].mean()]

# Round the averages to two decimal places
df_new['Avg Responses/Participant'] = df_new['Avg Responses/Participant'].round(2)
df_new['Avg Duration/Response'] = df_new['Avg Duration/Response'].round(2)

# Print as LaTeX table
latex_table = df_new.T.to_latex(index=True, caption="Summary Statistics of Experiment", label="overall_stats")
print(latex_table)


### Sources

In [ ]:
participants = df.groupby('source')['participant_id'].nunique()
percentage_participants = (participants / df['participant_id'].nunique()) * 100
trials = df['source'].value_counts()
percentage_trials = (trials / df.shape[0]) * 100
result = pd.DataFrame({
    'Participants': participants,
    'Percentage of Participants': percentage_participants,
    'Trials': trials,
    'Percentage of Trials': percentage_trials,
})

result = result.sort_values('Participants', ascending=False)
result['Participants (N, % of total)'] = result['Participants'].astype(str) + ' (' + result['Percentage of Participants'].round(1).astype(str) + '%)'
result['Trials (N, % of total)'] = result['Trials'].astype(str) + ' (' + result['Percentage of Trials'].round(1).astype(str) + '%)'
result = result.drop(columns=['Participants', 'Percentage of Participants', 'Trials', 'Percentage of Trials'])
latex_table = result.to_latex(caption="Sources of participants and trials", label="sources")

print(latex_table)


In [ ]:
import numpy as np
import pandas as pd

class DescriptiveStats:
    def __init__(self, df):
        self.df = df

    def generate_categorical_string(self, column, decimal_places):
        percentages = self.df[column].value_counts(normalize=True, dropna=True) * 100
        percentage_nan = self.df[column].isna().mean() * 100
        percentage_string = ", ".join([f"{percentage:.{decimal_places}f}\% {index}" for index, percentage in percentages.items()])
        percentage_string += f". {percentage_nan:.{decimal_places}f}% of responses were missing."
        return f"Of the non-missing values, the sample was {percentage_string}"

    def print_latex_table_categorical(self, column, caption=None, label=None, decimal_places=2):
        value_counts = self.df[column].value_counts(dropna=False)
        value_counts_percentages = self.df[column].value_counts(normalize=True, dropna=False) * 100
        df_output = pd.DataFrame({column: value_counts.index, 
                                  'Counts (% of total)': [f"{count} ({percentage:.{decimal_places}f}%)" for count, percentage in zip(value_counts.values, value_counts_percentages.values)]})
        df_output[column] = df_output[column].fillna("Missing")
        df_output.set_index(column, inplace=True)
        latex_table = df_output.to_latex(index=True, caption=caption or f"Distribution of {column.replace('_', ' ').title()}", label=label or f"dist_{column}")
        print(latex_table)
        print(self.generate_categorical_string(column, decimal_places))
    
    def generate_continuous_string(self, column, decimal_places):
        mean_value = round(self.df[column].mean(), decimal_places)
        std_dev = round(self.df[column].std(), decimal_places)
        stats_string = f"The mean {column} was {mean_value:.{decimal_places}f} (SD = {std_dev:.{decimal_places}f})."
        return stats_string

    def print_latex_table_continuous(self, columns, caption=None, label=None, decimal_places=2):
        if not isinstance(columns, list):
            columns = [columns]
        stats_list = []
        for column in columns:
            mean_value = round(self.df[column].mean(), decimal_places)
            median_value = round(self.df[column].median(), decimal_places)
            std_dev = round(self.df[column].std(), decimal_places)
            quartile_25 = round(np.percentile(self.df[column].dropna(), 25), decimal_places)
            quartile_75 = round(np.percentile(self.df[column].dropna(), 75), decimal_places)
            stats_series = pd.Series({'Mean': mean_value,
                                      'Median': median_value,
                                      'SD': std_dev,
                                      '25th Percentile': quartile_25,
                                      '75th Percentile': quartile_75},
                                     name=column)
            stats_list.append(stats_series)

        df_output = pd.concat(stats_list, axis=1)
        latex_table = df_output.T.to_latex(index=True, caption=caption or "Distribution of Continuous Variables", label=label)
        print(latex_table)
        for column in columns:
            print(self.generate_continuous_string(column, decimal_places))

ds = DescriptiveStats(df)
dp = DescriptiveStats(df.drop_duplicates(subset=['participant_id']))

dp.print_latex_table_continuous(['age', 'creativity_ai', 'creativity_human'], decimal_places=2, caption="Descriptive Stats")

dp.print_latex_table_categorical('gender', decimal_places=0)

dp.print_latex_table_categorical('ai_feeling', decimal_places=0)


# Race conditions

In [ ]:
df